In [4]:
import os
from pathlib import Path
from dotenv import load_dotenv

cwd = Path.cwd()
# Load dotenv into sys PATH
dotenv_file = cwd / "../../../example/.env"
assert dotenv_file.is_file()
load_dotenv(dotenv_file)

True

In [14]:
import pandas as pd
csv_file = r"../../../example/gbif_example\0026013-240906103802322\0026013-240906103802322.csv"
coff_dl_df = pd.read_csv(csv_file, sep='\t')

In [15]:
df = coff_dl_df.loc[coff_dl_df["taxonRank"] == "SPECIES"]
df = df.loc[:, ["verbatimScientificName", "decimalLongitude", "decimalLatitude", "year"]]
df = df.reset_index().drop(columns="index")
df["specimen_id"] = df.apply(lambda row: f"{row['verbatimScientificName']}_{row.name}", axis=1)
df["specimen_id"] = df["specimen_id"].str.strip("Coffea ")
df.columns = ["genus_species", "longitude", "latitude", "year", "specimen_id"]
df.head()

,genus_species,longitude,latitude,year,specimen_id
0,Psilanthus brassii,0.000000,0.000000,1967.0,Psilanthus brassii_0
1,Psilanthus brassii,147.333333,-9.333333,1967.0,Psilanthus brassii_1
2,Psilanthus brassii,146.500000,-8.783333,1933.0,Psilanthus brassii_2
3,Coffea arabica,0.000000,0.000000,1928.0,rabica_3
4,Coffea arabica Linnaeus,167.425000,-22.658333,1981.0,rabica Linnaeus_4


In [16]:
# Last 6 years
df_2018_p = df.loc[df["year"] > 2018]
print(len(df_2018_p))

3927


In [18]:
import sys
import os
import re

df_gbif = df_2018_p[['specimen_id', 'longitude', 'latitude']].copy()
df_node = pd.read_csv(r"..\..\input\node_names.csv")

#print(df_node)

# Function to extract part of the specimen_id for matching
def extract_name(specimen_id):
    # Use regex to extract pattern (e.g., everything before the underscore)
    return re.sub(r'_\d+', '', specimen_id)

    # Apply extraction function to both DataFrames
df_gbif['key'] = df_gbif['specimen_id'].apply(extract_name)
df_node['key'] = df_node['Node Name'].apply(lambda x: re.sub(r'^C_|_[\dA-Za-z]+$', '', x))

    # Create a dictionary for mapping key to Node Name
mapping = df_node.set_index('key')['Node Name'].to_dict()

    # Map the Node Name into a new column in df_gbif
df_gbif['Node Name'] = df_gbif['key'].map(mapping)



    # Drop the key column (optional)
df_gbif.drop(columns='key', inplace=True)
df_gbif = df_gbif.dropna(subset=['Node Name'])

    #print(df_gbif)


df_new = df_gbif[['Node Name','longitude', 'latitude']]

    # Renaming columns
df_new = df_new.rename(columns={'Node Name': 'specimen_id'})

    #print(df_new)
        
base_name, extension = os.path.splitext(csv_file)

formatted_csv_file = base_name + '_formatted' + extension
df_gbif.to_csv(formatted_csv_file, index=False)

print(f"Data saved to {formatted_csv_file}")

Data saved to ../../../example/gbif_example\0026013-240906103802322\0026013-240906103802322_formatted.csv


C:\Users\forc2101\AppData\Local\Temp\ipykernel_22608\1931540647.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gbif['key'] = df_gbif['specimen_id'].apply(extract_name)
